In [1]:
import tkinter as tk
from tkinter import messagebox
from abc import ABC, abstractmethod
from enum import Enum
import copy
from typing import List, Tuple, Optional

class PieceType(Enum):
    KING = "♔"
    QUEEN = "♕"
    ROOK = "♖"
    BISHOP = "♗"
    KNIGHT = "♘"
    PAWN = "♙"

class Color(Enum):
    WHITE = "white"
    BLACK = "black"

class Move:
    def __init__(self, start: Tuple[int, int], end: Tuple[int, int], piece_type: PieceType,
                 is_capture: bool = False, is_castle: bool = False,
                 is_promotion: bool = False, is_en_passant: bool = False):
        self.start = start
        self.end = end
        self.piece_type = piece_type
        self.is_capture = is_capture
        self.is_castle = is_castle
        self.is_promotion = is_promotion
        self.is_en_passant = is_en_passant

class Piece(ABC):
    def __init__(self, color: Color, piece_type: PieceType):
        self.color = color
        self.type = piece_type
        self.has_moved = False

    @abstractmethod
    def get_valid_moves(self, board: 'Board', pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        pass

    def get_symbol(self) -> str:
        return self.type.value

class Pawn(Piece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.PAWN)
        self.direction = 1 if color == Color.WHITE else -1  # Changed direction logic

    def get_valid_moves(self, board: 'Board', pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        moves = []
        row, col = pos

        # Forward move
        next_row = row + self.direction
        if 0 <= next_row < 8 and board.is_empty((next_row, col)):
            moves.append((next_row, col))
            # Initial two-square move
            if not self.has_moved:
                next_next_row = row + 2 * self.direction
                if 0 <= next_next_row < 8 and board.is_empty((next_next_row, col)):
                    moves.append((next_next_row, col))

        # Captures
        for dcol in [-1, 1]:
            new_col = col + dcol
            if 0 <= new_col < 8:
                capture_pos = (next_row, new_col)
                if board.is_valid_position(capture_pos):
                    piece = board.get_piece(capture_pos)
                    if piece and piece.color != self.color:
                        moves.append(capture_pos)

        return moves

class King(Piece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.KING)

    def get_valid_moves(self, board: 'Board', pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        moves = []
        row, col = pos
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

        for dr, dc in directions:
            new_row = row + dr
            new_col = col + dc
            if board.is_valid_position((new_row, new_col)):
                target_piece = board.get_piece((new_row, new_col))
                if target_piece is None or target_piece.color != self.color:
                    moves.append((new_row, new_col))

        return moves

class Queen(Piece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.QUEEN)

    def get_valid_moves(self, board: 'Board', pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        moves = []
        row, col = pos
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

        for dr, dc in directions:
            for i in range(1, 8):
                new_row = row + dr * i
                new_col = col + dc * i
                if not board.is_valid_position((new_row, new_col)):
                    break
                target_piece = board.get_piece((new_row, new_col))
                if target_piece is None:
                    moves.append((new_row, new_col))
                elif target_piece.color != self.color:
                    moves.append((new_row, new_col))
                    break
                else:
                    break

        return moves

class Rook(Piece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.ROOK)

    def get_valid_moves(self, board: 'Board', pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        moves = []
        row, col = pos
        directions = [(-1, 0), (0, -1), (0, 1), (1, 0)]

        for dr, dc in directions:
            for i in range(1, 8):
                new_row = row + dr * i
                new_col = col + dc * i
                if not board.is_valid_position((new_row, new_col)):
                    break
                target_piece = board.get_piece((new_row, new_col))
                if target_piece is None:
                    moves.append((new_row, new_col))
                elif target_piece.color != self.color:
                    moves.append((new_row, new_col))
                    break
                else:
                    break

        return moves

class Bishop(Piece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.BISHOP)

    def get_valid_moves(self, board: 'Board', pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        moves = []
        row, col = pos
        directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]

        for dr, dc in directions:
            for i in range(1, 8):
                new_row = row + dr * i
                new_col = col + dc * i
                if not board.is_valid_position((new_row, new_col)):
                    break
                target_piece = board.get_piece((new_row, new_col))
                if target_piece is None:
                    moves.append((new_row, new_col))
                elif target_piece.color != self.color:
                    moves.append((new_row, new_col))
                    break
                else:
                    break

        return moves

class Knight(Piece):
    def __init__(self, color: Color):
        super().__init__(color, PieceType.KNIGHT)

    def get_valid_moves(self, board: 'Board', pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        moves = []
        row, col = pos
        knight_moves = [(-2, -1), (-2, 1), (-1, -2), (-1, 2), (1, -2), (1, 2), (2, -1), (2, 1)]

        for dr, dc in knight_moves:
            new_row = row + dr
            new_col = col + dc
            if board.is_valid_position((new_row, new_col)):
                target_piece = board.get_piece((new_row, new_col))
                if target_piece is None or target_piece.color != self.color:
                    moves.append((new_row, new_col))

        return moves

class Board:
    @staticmethod
    def to_board_coords(algebraic: str) -> Tuple[int, int]:
        """Convert algebraic notation (e.g., 'e2') to board coordinates"""
        try:
            col = ord(algebraic[0].lower()) - ord('a')
            row = 8 - int(algebraic[1])
            if 0 <= row < 8 and 0 <= col < 8:
                return (row, col)
        except (IndexError, ValueError):
            return None
        return None

    @staticmethod
    def to_algebraic(row: int, col: int) -> str:
        """Convert board coordinates to algebraic notation"""
        return f"{chr(col + ord('a'))}{8 - row}"

    def __init__(self):
        self.board = [[None for _ in range(8)] for _ in range(8)]
        self.setup_board()

    def setup_board(self):
        # Set up pawns
        for col in range(8):
            self.board[1][col] = Pawn(Color.WHITE)
            self.board[6][col] = Pawn(Color.BLACK)
            self.board[0][col] = Rook(Color.WHITE)
            self.board[7][col] = Rook(Color.BLACK)
        self.board[0][1] = Knight(Color.WHITE)
        self.board[0][6] = Knight(Color.WHITE)
        self.board[7][1] = Knight(Color.BLACK)
        self.board[7][6] = Knight(Color.BLACK)
        self.board[0][2] = Bishop(Color.WHITE)
        self.board[0][5] = Bishop(Color.WHITE)
        self.board[7][2] = Bishop(Color.BLACK)
        self.board[7][5] = Bishop(Color.BLACK)
        self.board[0][3] = Queen(Color.WHITE)
        self.board[0][4] = King(Color.WHITE)
        self.board[7][3] = Queen(Color.BLACK)
        self.board[7][4] = King(Color.BLACK)

    def is_valid_position(self, pos: Tuple[int, int]) -> bool:
        row, col = pos
        return 0 <= row < 8 and 0 <= col < 8

    def is_empty(self, pos: Tuple[int, int]) -> bool:
        return self.get_piece(pos) is None

    def get_piece(self, pos: Tuple[int, int]) -> Optional[Piece]:
        row, col = pos
        return self.board[row][col]

    def make_move(self, move: Move) -> None:
        piece = self.get_piece(move.start)
        self.board[move.end[0]][move.end[1]] = piece
        self.board[move.start[0]][move.start[1]] = None
        piece.has_moved = True

    def is_in_check(self, color: Color) -> bool:
        # Find the king's position
        king_pos = None
        for row in range(8):
            for col in range(8):
                piece = self.get_piece((row, col))
                if piece and piece.type == PieceType.KING and piece.color == color:
                    king_pos = (row, col)
                    break
            if king_pos:
                break

        # Check if any opponent's piece can capture the king
        opponent_color = Color.BLACK if color == Color.WHITE else Color.WHITE
        for row in range(8):
            for col in range(8):
                piece = self.get_piece((row, col))
                if piece and piece.color == opponent_color:
                    moves = piece.get_valid_moves(self, (row, col))
                    if king_pos in moves:
                        return True
        return False

    def is_checkmate(self, color: Color) -> bool:
        if not self.is_in_check(color):
            return False

        # Try all possible moves for all pieces of the given color
        for row in range(8):
            for col in range(8):
                piece = self.get_piece((row, col))
                if piece and piece.color == color:
                    moves = piece.get_valid_moves(self, (row, col))
                    for move in moves:
                        # Try the move
                        temp_board = copy.deepcopy(self)
                        temp_board.make_move(Move((row, col), move, piece.type))
                        # If the move gets us out of check, it's not checkmate
                        if not temp_board.is_in_check(color):
                            return False
        return True

    def get_valid_moves_considering_check(self, pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        piece = self.get_piece(pos)
        if not piece:
            return []

        moves = piece.get_valid_moves(self, pos)
        valid_moves = []

        # Test each move to see if it leaves the king in check
        for move in moves:
            temp_board = copy.deepcopy(self)
            temp_board.make_move(Move(pos, move, piece.type))
            if not temp_board.is_in_check(piece.color):
                valid_moves.append(move)

        return valid_moves

class AIPlayer:
    def __init__(self, color: Color):
        self.color = color
        self.max_depth = 3

    def get_move(self, board: Board) -> Move:
        _, best_move = self.minimax(board, self.max_depth, float('-inf'), float('inf'), True)
        return best_move

    def minimax(self, board: Board, depth: int, alpha: float, beta: float, maximizing: bool) -> Tuple[float, Optional[Move]]:
        if depth == 0:
            return self.evaluate_position(board), None

        best_move = None
        if maximizing:
            max_eval = float('-inf')
            for row in range(8):
                for col in range(8):
                    piece = board.get_piece((row, col))
                    if piece and piece.color == self.color:
                        moves = piece.get_valid_moves(board, (row, col))
                        for move in moves:
                            new_board = copy.deepcopy(board)
                            new_board.make_move(Move((row, col), move, piece.type))
                            eval = self.minimax(new_board, depth - 1, alpha, beta, False)[0]
                            if eval > max_eval:
                                max_eval = eval
                                best_move = Move((row, col), move, piece.type)
                            alpha = max(alpha, eval)
                            if beta <= alpha:
                                break
            return max_eval, best_move
        else:
            min_eval = float('inf')
            for row in range(8):
                for col in range(8):
                    piece = board.get_piece((row, col))
                    if piece and piece.color != self.color:
                        moves = piece.get_valid_moves(board, (row, col))
                        for move in moves:
                            new_board = copy.deepcopy(board)
                            new_board.make_move(Move((row, col), move, piece.type))
                            eval = self.minimax(new_board, depth - 1, alpha, beta, True)[0]
                            if eval < min_eval:
                                min_eval = eval
                                best_move = Move((row, col), move, piece.type)
                            beta = min(beta, eval)
                            if beta <= alpha:
                                break
            return min_eval, best_move

    def evaluate_position(self, board: Board) -> float:
        score = 0.0
        piece_values = {
            PieceType.KING: 200.0,
            PieceType.QUEEN: 9.0,
            PieceType.ROOK: 5.0,
            PieceType.BISHOP: 3.0,
            PieceType.KNIGHT: 3.0,
            PieceType.PAWN: 1.0
        }

        # Check for checkmate
        if board.is_checkmate(self.color):
            return float('-inf')
        elif board.is_checkmate(Color.BLACK if self.color == Color.WHITE else Color.WHITE):
            return float('inf')

        # Check for check
        if board.is_in_check(self.color):
            score -= 5.0
        if board.is_in_check(Color.BLACK if self.color == Color.WHITE else Color.WHITE):
            score += 5.0

        # Material evaluation
        for row in range(8):
            for col in range(8):
                piece = board.get_piece((row, col))
                if piece:
                    value = piece_values[piece.type]
                    if piece.color == self.color:
                        score += value
                    else:
                        score -= value

        # Positional evaluation - center control
        center_squares = [(3,3), (3,4), (4,3), (4,4)]
        for square in center_squares:
            piece = board.get_piece(square)
            if piece:
                if piece.color == self.color:
                    score += 0.3
                else:
                    score -= 0.3

        # King safety - count defender pieces around king
        def find_king(color):
            for row in range(8):
                for col in range(8):
                    piece = board.get_piece((row, col))
                    if piece and piece.type == PieceType.KING and piece.color == color:
                        return (row, col)
            return None

        king_pos = find_king(self.color)
        enemy_king_pos = find_king(Color.BLACK if self.color == Color.WHITE else Color.WHITE)

        if king_pos:
            # Check surrounding squares for defenders
            king_row, king_col = king_pos
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    if dr == 0 and dc == 0:
                        continue
                    check_pos = (king_row + dr, king_col + dc)
                    if board.is_valid_position(check_pos):
                        piece = board.get_piece(check_pos)
                        if piece and piece.color == self.color:
                            score += 0.2  # Bonus for each defending piece

        # Penalize exposed king
        if enemy_king_pos:
            if enemy_king_pos[1] in [0, 7]:  # King on edge
                score += 0.5

        return score

class ChessGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Chess Game")
        self.board = Board()
        self.ai = AIPlayer(Color.BLACK)
        self.selected_piece = None
        self.valid_moves = []
        self.setup_gui()

    # def setup_gui(self):
    #     # Main container
    #     main_frame = tk.Frame(self.root)
    #     main_frame.pack(expand=True, fill=tk.BOTH)

    #     # Left frame for the chess board
    #     left_frame = tk.Frame(main_frame)
    #     left_frame.pack(side=tk.LEFT, padx=10, pady=10)

    #     # Right frame for input and move history
    #     right_frame = tk.Frame(main_frame)
    #     right_frame.pack(side=tk.LEFT, padx=10, pady=10)

    #     # Chess board canvas
    #     self.canvas = tk.Canvas(left_frame, width=400, height=400)
    #     self.canvas.pack()

    #     # Move input frame
    #     input_frame = tk.Frame(right_frame)
    #     input_frame.pack(fill=tk.X, pady=(0, 10))

    #     # Move input label and entry
    #     tk.Label(input_frame, text="Enter move (e.g., e2e4):").pack()
    #     self.move_entry = tk.Entry(input_frame)
    #     self.move_entry.pack(fill=tk.X, pady=(0, 5))
    #     self.move_entry.bind('<Return>', self.process_move_input)

    #     # Submit button
    #     tk.Button(input_frame, text="Make Move", command=self.process_move_input).pack()

    #     # Initialize the board
    #     self.draw_board()
    #     self.canvas.bind("<Button-1>", self.on_click)

    def setup_gui(self):
        # Set background color for the main window
        self.root.configure(bg="#2b1810")  # Dark brown background

        # Main container with matching background
        main_frame = tk.Frame(self.root, bg="#2b1810")
        main_frame.pack(expand=True, fill=tk.BOTH, padx=20, pady=20)

        # Left frame for the chess board
        left_frame = tk.Frame(main_frame, bg="#2b1810")
        left_frame.pack(side=tk.LEFT, padx=10, pady=10)

        # Right frame for input and move history
        right_frame = tk.Frame(main_frame, bg="#2b1810")
        right_frame.pack(side=tk.LEFT, padx=10, pady=10)

        # Chess board canvas with border effect
        canvas_frame = tk.Frame(left_frame, bg="#8B4513", padx=3, pady=3)  # Brown border
        canvas_frame.pack()
        self.canvas = tk.Canvas(canvas_frame, width=400, height=400)
        self.canvas.pack()

        # Move input frame with matching background
        input_frame = tk.Frame(right_frame, bg="#2b1810")
        input_frame.pack(fill=tk.X, pady=(0, 10))

        # Move input label with contrasting color
        tk.Label(input_frame,
                text="Enter move (e.g., e2e4):",
                bg="#2b1810",
                fg="white",
                font=('Arial', 10)).pack()

        self.move_entry = tk.Entry(input_frame)
        self.move_entry.pack(fill=tk.X, pady=(5, 5))
        self.move_entry.bind('<Return>', self.process_move_input)

        # Submit button with matching style
        tk.Button(input_frame,
                text="Make Move",
                command=self.process_move_input,
                bg="#8B4513",
                fg="white",
                activebackground="#654321",
                activeforeground="white").pack(pady=5)

        # Initialize the board
        self.draw_board()
        self.canvas.bind("<Button-1>", self.on_click)

    def process_move_input(self, event=None):
        """Process move input from the entry field"""
        move_text = self.move_entry.get().strip().lower()
        self.move_entry.delete(0, tk.END)

        # Handle different input formats
        if ' ' in move_text:  # Format: "e2 e4"
            start, end = move_text.split()
        elif len(move_text) == 4:  # Format: "e2e4"
            start, end = move_text[:2], move_text[2:]
        else:
            messagebox.showerror("Invalid Input", "Please enter move in format 'e2e4' or 'e2 e4'")
            return

        # Convert to board coordinates
        start_pos = self.board.to_board_coords(start)
        end_pos = self.board.to_board_coords(end)

        if not (start_pos and end_pos):
            messagebox.showerror("Invalid Move", "Invalid square notation")
            return

        # Validate the move
        piece = self.board.get_piece(start_pos)
        if not piece or piece.color != Color.WHITE:
            messagebox.showerror("Invalid Move", "No valid piece at starting position")
            return

        valid_moves = self.board.get_valid_moves_considering_check(start_pos)
        if end_pos not in valid_moves:
            messagebox.showerror("Invalid Move", "Invalid move for this piece")
            return

        # Make the move
        move = Move(start_pos, end_pos, piece.type)
        self.board.make_move(move)
        self.draw_board()

        # Check for check/checkmate and make AI move
        self.handle_post_move()

    def draw_board(self):
        colors = ["#d6c089", "#34612f"]  # Board colors
        piece_colors = {
            Color.WHITE: "#FFE5CC",  # Light cream color for white pieces
            Color.BLACK: "#4A2100"   # Dark brown color for black pieces
        }

        # Add padding for coordinates
        padding = 25

        # Clear the canvas and resize it to accommodate coordinates
        self.canvas.config(width=400 + 2*padding, height=400 + 2*padding)
        self.canvas.delete("all")

        # Draw coordinates
        for i in range(8):
            # Draw file coordinates (a-h)
            self.canvas.create_text(
                padding + i*50 + 25,  # x position (centered in square)
                padding - 18,        # y position (below board)
                text=chr(97 + i),     # 'a' through 'h'
                font=('Arial', 12)
            )
            # Draw rank coordinates (1-8)
            self.canvas.create_text(
                5,                    # x position (left of board)
                padding + (7-i)*50 + 25,  # y position (centered in square)
                text=str(i + 1),
                font=('Arial', 12)
            )

        # Draw board squares and pieces
        for row in range(8):
            for col in range(8):
                # Draw board squares
                color = colors[(row + col) % 2]
                self.canvas.create_rectangle(
                    padding + col*50,
                    padding + row*50,
                    padding + (col+1)*50,
                    padding + (row+1)*50,
                    fill=color,
                    outline=""
                )

                # Draw pieces
                piece = self.board.get_piece((row, col))
                if piece:
                    piece_color = piece_colors[piece.color]
                    self.canvas.create_text(
                        padding + col*50 + 25,
                        padding + row*50 + 25,
                        text=piece.get_symbol(),
                        fill=piece_color,
                        font=('Arial', 24)
                    )

    def handle_post_move(self):
        """Handle post-move actions (check, checkmate, AI move)"""
        # Check if Black is in checkmate or check after White's move
        if self.board.is_checkmate(Color.BLACK):
            messagebox.showinfo("Game Over", "Checkmate! White wins!")
            self.root.quit()
            return
        elif self.board.is_in_check(Color.BLACK):
            messagebox.showinfo("Check", "Black is in check!")

        # Show "AI is thinking..." message
        thinking_text = self.canvas.create_text(200, 200,
                                          text="AI is thinking...",
                                          fill="white",
                                          font=('Arial', 20))
        self.root.update()

        # AI move
        ai_move = self.ai.get_move(self.board)
        if ai_move:
            self.board.make_move(ai_move)
            self.draw_board()

            # Check if White is in checkmate or check after Black's move
            if self.board.is_checkmate(Color.WHITE):
                messagebox.showinfo("Game Over", "Checkmate! Black wins!")
                self.root.quit()
            elif self.board.is_in_check(Color.WHITE):
                messagebox.showinfo("Check", "White is in check!")

        self.canvas.delete(thinking_text)

    def on_click(self, event):
        padding = 25
        # Adjust for padding in coordinate calculation
        col = (event.x - padding) // 50
        row = (event.y - padding) // 50

        # Check if click is within the board boundaries
        if 0 <= row < 8 and 0 <= col < 8:
            if not self.selected_piece:
                piece = self.board.get_piece((row, col))
                if piece and piece.color == Color.WHITE:
                    self.selected_piece = (row, col)
                    valid_moves = self.board.get_valid_moves_considering_check((row, col))
                    self.highlight_moves(valid_moves)
            else:
                if (row, col) in self.valid_moves:
                    move = Move(self.selected_piece, (row, col),
                            self.board.get_piece(self.selected_piece).type)
                    self.board.make_move(move)
                    self.selected_piece = None
                    self.draw_board()
                    self.handle_post_move()
                else:
                    self.selected_piece = None
                    self.draw_board()

    # def on_click(self, event):
    #     col = event.x // 50
    #     row = event.y // 50

    #     if not self.selected_piece:
    #         piece = self.board.get_piece((row, col))
    #         if piece and piece.color == Color.WHITE:
    #             self.selected_piece = (row, col)
    #             # Use get_valid_moves_considering_check instead of get_valid_moves
    #             valid_moves = self.board.get_valid_moves_considering_check((row, col))
    #             self.highlight_moves(valid_moves)
    #     else:
    #         if (row, col) in self.valid_moves:
    #             move = Move(self.selected_piece, (row, col),
    #                       self.board.get_piece(self.selected_piece).type)
    #             self.board.make_move(move)
    #             self.selected_piece = None
    #             self.draw_board()
    #             self.handle_post_move()

    #             # Check if Black is in checkmate or check after White's move
    #             if self.board.is_checkmate(Color.BLACK):
    #                 messagebox.showinfo("Game Over", "Checkmate! White wins!")
    #                 self.root.quit()
    #                 return
    #             elif self.board.is_in_check(Color.BLACK):
    #                 messagebox.showinfo("Check", "Black is in check!")

    #             # Show "AI is thinking..." message
    #             thinking_text = self.canvas.create_text(200, 200,
    #                                               text="AI is thinking...",
    #                                               fill="white",
    #                                               font=('Arial', 20))
    #             self.root.update()

    #             # AI move
    #             ai_move = self.ai.get_move(self.board)
    #             if ai_move:
    #                 self.board.make_move(ai_move)
    #                 self.draw_board()

    #                 # Check if White is in checkmate or check after Black's move
    #                 if self.board.is_checkmate(Color.WHITE):
    #                     messagebox.showinfo("Game Over", "Checkmate! Black wins!")
    #                     self.root.quit()
    #                 elif self.board.is_in_check(Color.WHITE):
    #                     messagebox.showinfo("Check", "White is in check!")

    #             self.canvas.delete(thinking_text)
    #         else:
    #             self.selected_piece = None
    #             self.draw_board()

    def highlight_moves(self, moves):
        padding = 25
        self.valid_moves = moves
        for move in moves:
            row, col = move
            self.canvas.create_oval(
                padding + col*50 + 15,
                padding + row*50 + 15,
                padding + col*50 + 35,
                padding + row*50 + 35,
                fill="green",
                stipple="gray50"
            )

    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    game = ChessGUI()
    game.run()